In [ ]:
import coiled
import dask.distributed
import dask.dataframe as dd

In [ ]:
cluster = coiled.Cluster(configuration="coiled/default", n_workers=5)

In [ ]:
client = dask.distributed.Client(cluster)

In [40]:
d2000 = dd.read_parquet("s3://coiled-datasets/timeseries/7d/parquet/2000")
d2001 = dd.read_parquet("s3://coiled-datasets/timeseries/7d/parquet/2001")
d2002 = dd.read_parquet("s3://coiled-datasets/timeseries/7d/parquet/2002")
d2003 = dd.read_parquet("s3://coiled-datasets/timeseries/7d/parquet/2003")
d2004 = dd.read_parquet("s3://coiled-datasets/timeseries/7d/parquet/2004")

In [41]:
ddf = dd.concat([d2000, d2001, d2002, d2003, d2004])

In [42]:
len(ddf)

OSError: [Errno 22] Bad Request

In [28]:
ddf.npartitions

260

## pyarrow write setup

In [30]:
import pyarrow as pa

In [31]:
schema={"id": pa.uint32(), "name": pa.string(), "x": pa.float64(), "y": pa.float64()}

## noop write time

In [29]:
%%time
ddf.to_parquet("s3://coiled-datasets/tmp/shuffle/dask/noop", write_index=False)

CPU times: user 216 ms, sys: 33.4 ms, total: 249 ms
Wall time: 48 s


In [38]:
%%time
ddf.to_parquet("s3://coiled-datasets/tmp/shuffle/dask/noop2", write_index=False, schema=schema)

CPU times: user 238 ms, sys: 21.4 ms, total: 259 ms
Wall time: 48.2 s


## shuffle times without writes

In [32]:
%%time
sorted_ddf_id = ddf.set_index("id")

CPU times: user 754 ms, sys: 73 ms, total: 827 ms
Wall time: 28.2 s


In [33]:
sorted_ddf_id.head()

,name,x,y
id,,,
832,Oliver,-0.015893,0.678568
832,Oliver,-0.015893,0.678568
832,Oliver,-0.015893,0.678568
832,Oliver,-0.015893,0.678568
832,Oliver,-0.015893,0.678568


In [34]:
sorted_ddf_id.npartitions

260

## shuffle write time

In [35]:
%%time

ddf.set_index("id").to_parquet("s3://coiled-datasets/tmp/shuffle/dask/shuffled", schema=schema)

CPU times: user 1.65 s, sys: 167 ms, total: 1.81 s
Wall time: 1min 40s


## shuffle and repartition write time

In [37]:
%%time
ddf.set_index("id").repartition(50).to_parquet("s3://coiled-datasets/tmp/shuffle/dask/shuf_and_part", schema=schema)

CPU times: user 1.64 s, sys: 137 ms, total: 1.77 s
Wall time: 1min 57s
